# Pip Install

In [1]:
!pip install boto3

In [2]:
!pip install astropy

In [3]:
!pip install sfdmap

In [4]:
!pip install progressbar2

# Initialization

In [5]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
import boto3
from matplotlib import pyplot as plt

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'thesis2')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3-west.nrp-nautilus.io'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'thesis2')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_final_table_csv_path = os.path.join(s3_data_dir_path, 'SDSS_DR16_all.csv')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig, from_s3_csv

2023-04-01 14:16:55.417154: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 14:16:56.694613: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-01 14:16:56.694710: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-04-01 14:16:56.694723: W 

# Load the Dataset Info

In [6]:
# loading the table
final_table_csv_path = os.path.join(endpoint_url,bucket_name,s3_final_table_csv_path)
gs = pd.read_csv(final_table_csv_path, header = 0, dtype = {
    'specobjid' : str,
    'z' : float,
    'z_noqso' : float,
    'snMedian' : float,
    'run2d' : str,
    'ra' : float,
    'dec' : float,
    'plate' : int,
    'mjd' : int,
    'fiberid' : int,
    'waveMin' : float,
    'waveMax' : float
    })
print('Total galaxies in DR16: %d' % len(gs))

Total galaxies in DR16: 3234563


In [7]:
gs['key'] = ['{0}_{1}_{2}'.format(p,m,f) for p,m,f in zip(gs['plate'], gs['mjd'], gs['fiberid'])]

## Load Dalya's Outliers

In [8]:
# load the table
dalya_outliers = from_s3_csv(s3_client, bucket_name, 'almogh/thesis2/data/Dalya_outlier_classes.csv')
dalya_outliers['key'] = ['{0}_{1}_{2}'.format(p,m,f) for p,m,f in zip(dalya_outliers['plate'], dalya_outliers['mjd'], dalya_outliers['fiberid'])]

# remove non-galaxy
dalya_outliers = dalya_outliers[dalya_outliers['class']=='GALAXY']

# get ourliers from the full table
gs_outliers = gs[gs['key'].isin(dalya_outliers['key'].unique())]

# make sure no outlier was missed
assert len(gs_outliers)==len(dalya_outliers['key'].unique()), "some of Dalya's outliers are not listed!"

print('total outliers: {0}. total tags: {1}'.format(len(dalya_outliers['key'].unique()), len(dalya_outliers)))

loading from uri: s3://tau-astro/almogh/thesis2/data/Dalya_outlier_classes.csv
total outliers: 360. total tags: 394


# Filter and Merge with other Data Sources

## Filter by SNR and Z

In [9]:
# parameters
snr_th = 0.6
z_th = 0.03

In [10]:
# init counter
outliers_cnt = len(gs_outliers)
outliers_total_filtered = 0

# Remove galaxies with low redshift
gs_outliers = gs_outliers[gs_outliers.z > z_th]
gs_outliers.index = np.arange(len(gs_outliers))
outliers_filtered = outliers_cnt-len(gs_outliers)
print('Total galaxies filtered with low z values: %d' % outliers_filtered)
outliers_cnt = len(gs_outliers)
outliers_total_filtered += outliers_filtered

# Remove galaxies with low SNR
gs_outliers = gs_outliers[gs_outliers.snMedian > snr_th]
gs_outliers.index = np.arange(len(gs_outliers))
outliers_filtered = outliers_cnt-len(gs_outliers)
print('Total galaxies filtered with low snr values: %d' % outliers_filtered)
outliers_cnt = len(gs_outliers)
outliers_total_filtered += outliers_filtered

Total galaxies filtered with low z values: 120
Total galaxies filtered with low snr values: 0


## Merge with Dalya's Scores

In [11]:
# adding Dalya's scores
dalya_scores_URL = 'http://wise-obs.tau.ac.il/~dovip/weird-galaxies/full.csv'
dalya_df = pd.read_csv(dalya_scores_URL)
dalya_df = dalya_df.rename(columns={" mjd":"mjd"," fiber":"fiberid", " weirdness":"weirdness"})

In [12]:
gs_outliers = gs_outliers.merge(dalya_df, how='inner', on=['plate', 'mjd', 'fiberid'])
gs_outliers.index = np.arange(len(gs_outliers))
outliers_filtered = outliers_cnt-len(gs_outliers)
print("Total galaxies filtered after merging with Dalya's scores: %d" % outliers_filtered)
outliers_cnt = len(gs_outliers)
outliers_total_filtered += outliers_filtered

Total galaxies filtered after merging with Dalya's scores: 13


## Merge with galSpecLine and galSpecInfo

In [16]:
# Merge with galSpecInfo

galSpecInfo = pd.read_csv(os.path.join(endpoint_url, bucket_name, s3_data_dir_path, 'galSpecInfo.csv'), header = 0, dtype = {
    'specobjid' : str,
    'plateid' : int,
    'mjd' : int,
    'fiberid' : int,
    'spectrotype' : str,
    'subclass' : str
    })
galSpecInfo = galSpecInfo.rename(columns={"plateid":"plate"})

In [14]:
# Merge with galSpecLine

galSpecLine = pd.read_csv(os.path.join(endpoint_url, bucket_name, s3_data_dir_path, 'galSpecLine.csv'), header = 0, dtype = {
    'specobjid' : str,
    'oii_3726_eqw' : float,
    'oii_3729_eqw' : float,
    'neiii_3869_eqw' : float,
    'h_delta_eqw' : float,
    'h_gamma_eqw' : float,
    'oiii_4363_eqw' : float,
    'h_beta_eqw' : float,
    'oiii_4959_eqw' : float,
    'oiii_5007_eqw' : float,
    'hei_5876_eqw' : float,
    'oi_6300_eqw' : float,
    'nii_6548_eqw' : float,
    'h_alpha_eqw' : float,
    'nii_6584_eqw' : float,
    'sii_6717_eqw' : float,
    'sii_6731_eqw' : float,
    'ariii7135_eqw' : float
    })

In [17]:
gs_outliers = gs_outliers.merge(galSpecInfo, how='inner', on=['specobjid', 'plate', 'mjd', 'fiberid'])
gs_outliers.index = np.arange(len(gs_outliers))
outliers_filtered = outliers_cnt-len(gs_outliers)
print("Total galaxies filtered after merging with galSpecInfo: %d" % outliers_filtered)
outliers_cnt = len(gs_outliers)
outliers_total_filtered += outliers_filtered

Total galaxies filtered after merging with galSpecInfo: 55


In [18]:
gs_outliers = gs_outliers.merge(galSpecLine, how='inner', on=['specobjid'])
gs_outliers.index = np.arange(len(gs_outliers))
outliers_filtered = outliers_cnt-len(gs_outliers)
print("Total galaxies filtered after merging with galSpecLine: %d" % outliers_filtered)
outliers_cnt = len(gs_outliers)
outliers_total_filtered += outliers_filtered

Total galaxies filtered after merging with galSpecLine: 0


## Filter by wavelength

In [19]:
wl_grid = np.arange(3800, 8000, 0.5)

In [20]:
gs_outliers['waveMin_rest'] = np.divide(gs_outliers.waveMin,1+gs_outliers.z)
gs_outliers['waveMax_rest'] = np.divide(gs_outliers.waveMax,1+gs_outliers.z)
gs_outliers_filtered = gs_outliers[(gs_outliers['waveMin_rest']<wl_grid[0])&(gs_outliers['waveMax_rest']>wl_grid[-1])]
gs_outliers_filtered.index = np.arange(len(gs_outliers_filtered))
print('Total number of galaxies left: %d' % len(gs_outliers_filtered))

Total number of galaxies left: 114


# Create the outliers datasets for the models

In [21]:
gs_NN_train = from_s3_pkl(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = 'almogh/thesis2/data/NN/train/gs.pkl')
gs_SmallRF_train = from_s3_pkl(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = 'almogh/thesis2/data/SmallRF/train/gs.pkl')
gs_BigRF_train = from_s3_pkl(s3_client = s3_client, bucket_name = bucket_name, path_in_bucket = 'almogh/thesis2/data/BigRF/train/gs.pkl')

loading from uri: s3://tau-astro/almogh/thesis2/data/NN/train/gs.pkl
loading from uri: s3://tau-astro/almogh/thesis2/data/SmallRF/train/gs.pkl
loading from uri: s3://tau-astro/almogh/thesis2/data/BigRF/train/gs.pkl


In [24]:
gs_outliers_filtered['in NN train'] = gs_outliers_filtered.specobjid.isin(gs_NN_train.specobjid)
gs_outliers_filtered['in SmallRF train'] = gs_outliers_filtered.specobjid.isin(gs_SmallRF_train.specobjid)
gs_outliers_filtered['in BigRF train'] = gs_outliers_filtered.specobjid.isin(gs_BigRF_train.specobjid)

/tmp/ipykernel_218/956641818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_outliers_filtered['in NN train'] = gs_outliers_filtered.specobjid.isin(gs_NN_train.specobjid)
/tmp/ipykernel_218/956641818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gs_outliers_filtered['in SmallRF train'] = gs_outliers_filtered.specobjid.isin(gs_SmallRF_train.specobjid)
/tmp/ipykernel_218/956641818.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [28]:
print('out of {0} filtered outliers, {1} are in BigRF train set, {2} are in SmallRF train set, {3} are in NN train set'.format(len(gs_outliers_filtered), sum(gs_outliers_filtered['in BigRF train']),sum(gs_outliers_filtered['in SmallRF train']),sum(gs_outliers_filtered['in NN train'])))

out of 114 filtered outliers, 30 are in BigRF train set, 8 are in SmallRF train set, 8 are in NN train set


# Download and pre-process the data

In [29]:
# create a wrapper that returns the index also
from pre_processing import download_spectrum
def download_spectrum_wrapper(i):
    spec, _, ivar = download_spectrum(gs_outliers_filtered.iloc[i], pre_proc=True, wl_grid=wl_grid)
    return i, spec, ivar

In [30]:
# create jobs to download and preprocess
from joblib import Parallel, delayed
res = Parallel(n_jobs=-1, verbose=5, prefer="threads")(delayed(download_spectrum_wrapper)(i) for i in range(len(gs_outliers_filtered)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  74 out of 114 | elapsed:   30.3s remaining:   16.4s
[Parallel(n_jobs=-1)]: Done  97 out of 114 | elapsed:   31.1s remaining:    5.5s
[Parallel(n_jobs=-1)]: Done 114 out of 114 | elapsed:   31.7s finished


In [31]:
# make sure all the data was successfully downloaded (exception encountered during download will return empty arrays)
assert all([len(val[1]) > 0 for val in res])

In [32]:
# organize the results to matrices
res = sorted(res, key=lambda x: x[0])
spec = np.stack([x[1] for x in res], axis=0)
ivar = np.stack([x[2] for x in res], axis=0)

# Save dataset

In [33]:
to_s3_pkl(gs_outliers_filtered, s3_client, bucket_name, os.path.join(s3_data_dir_path,'Outliers','gs.pkl'))
to_s3_npy(spec, s3_client, bucket_name, os.path.join(s3_data_dir_path,'Outliers','spec.npy'))
to_s3_npy(ivar, s3_client, bucket_name, os.path.join(s3_data_dir_path,'Outliers','ivar.npy'))
to_s3_npy(wl_grid, s3_client, bucket_name, os.path.join(s3_data_dir_path,'Outliers','wl_grid.npy'))

saving to uri: s3://tau-astro/almogh/thesis2/data/Outliers/gs.pkl
saving to uri: s3://tau-astro/almogh/thesis2/data/Outliers/spec.npy
saving to uri: s3://tau-astro/almogh/thesis2/data/Outliers/ivar.npy
saving to uri: s3://tau-astro/almogh/thesis2/data/Outliers/wl_grid.npy


True